# 03 - Geospatial Features

This notebook adds geospatial features:
- **Elevation** from SRTM DEM
- **Slope and Aspect** from terrain analysis
- **Land Cover** from ESA WorldCover
- **Spatial Clusters** using K-means
- **Distance Features** to reference points

These features capture topographic and land use patterns that influence water quality.

In [ ]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from data_loading import load_processed_data, save_processed_data
from geospatial_processing import (
    load_srtm_elevation,
    compute_terrain_features,
    load_esa_worldcover,
    create_spatial_clusters,
    create_distance_features,
    create_watershed_features
)

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
sns.set_style('whitegrid')

## 1. Load Engineered Data

In [ ]:
# Load engineered datasets from previous notebook
train = load_processed_data('../data/processed/train_engineered.parquet')
test = load_processed_data('../data/processed/test_engineered.parquet')

print(f"Training data: {train.shape}")
print(f"Test data: {test.shape}")

## 2. Add Elevation Data (SRTM DEM)

**Note**: This requires SRTM DEM GeoTIFF files. If not available locally, the code will create dummy features.

In [ ]:
# Path to SRTM DEM (update with actual path)
srtm_path = '../data/external/srtm_dem.tif'

# Check if DEM exists
import os
if os.path.exists(srtm_path):
    train = load_srtm_elevation(train, srtm_path)
    test = load_srtm_elevation(test, srtm_path)
    print("\nElevation data loaded from SRTM DEM")
else:
    print(f"\nSRTM DEM not found at {srtm_path}")
    print("Creating synthetic elevation features for demonstration...")
    
    # Create synthetic elevation based on lat/lon
    if 'latitude' in train.columns and 'longitude' in train.columns:
        train['elevation'] = (train['latitude'] * 10 + train['longitude'] * 5 + 
                             np.random.randn(len(train)) * 50 + 500)
        test['elevation'] = (test['latitude'] * 10 + test['longitude'] * 5 + 
                            np.random.randn(len(test)) * 50 + 500)
        print("Synthetic elevation features created")

## 3. Compute Terrain Features (Slope, Aspect)

In [ ]:
# Compute slope and aspect from DEM
dem_path = '../data/external/srtm_dem.tif'

if os.path.exists(dem_path):
    train = compute_terrain_features(train, dem_path)
    test = compute_terrain_features(test, dem_path)
    print("\nTerrain features computed")
else:
    print("DEM not available. Creating synthetic slope features...")
    
    # Calculate simple slope from elevation gradient
    if 'elevation' in train.columns:
        train['slope'] = np.abs(np.gradient(train['elevation'])) * 10
        test['slope'] = np.abs(np.gradient(test['elevation'])) * 10
        train['aspect'] = np.random.uniform(0, 360, len(train))
        test['aspect'] = np.random.uniform(0, 360, len(test))
        print("Synthetic slope and aspect created")

In [ ]:
# Visualize elevation distribution
if 'elevation' in train.columns and 'target' in train.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    axes[0].hist(train['elevation'], bins=50, edgecolor='black')
    axes[0].set_xlabel('Elevation (m)')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title('Elevation Distribution')
    
    axes[1].scatter(train['elevation'], train['target'], alpha=0.3, s=1)
    axes[1].set_xlabel('Elevation (m)')
    axes[1].set_ylabel('Target')
    axes[1].set_title('Elevation vs Target')
    
    plt.tight_layout()
    plt.savefig('../outputs/figures/elevation_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()

## 4. Add Land Cover Data (ESA WorldCover)

In [ ]:
# Path to ESA WorldCover (update with actual path)
worldcover_path = '../data/external/esa_worldcover.tif'

if os.path.exists(worldcover_path):
    train = load_esa_worldcover(train, worldcover_path)
    test = load_esa_worldcover(test, worldcover_path)
    print("\nLand cover data loaded")
else:
    print(f"\nESA WorldCover not found at {worldcover_path}")
    print("Creating synthetic land cover features...")
    
    # Create synthetic land cover classes
    land_cover_classes = [10, 20, 30, 40, 50, 60, 70, 80, 90]
    train['land_cover'] = np.random.choice(land_cover_classes, size=len(train))
    test['land_cover'] = np.random.choice(land_cover_classes, size=len(test))
    print("Synthetic land cover created")

In [ ]:
# Analyze land cover distribution
if 'land_cover' in train.columns:
    landcover_mapping = {
        10: 'Tree cover',
        20: 'Shrubland',
        30: 'Grassland',
        40: 'Cropland',
        50: 'Built-up',
        60: 'Bare/sparse',
        70: 'Snow/ice',
        80: 'Water',
        90: 'Wetland'
    }
    
    plt.figure(figsize=(12, 5))
    
    land_cover_counts = train['land_cover'].value_counts().sort_index()
    labels = [landcover_mapping.get(lc, str(lc)) for lc in land_cover_counts.index]
    
    plt.bar(range(len(land_cover_counts)), land_cover_counts.values)
    plt.xticks(range(len(land_cover_counts)), labels, rotation=45, ha='right')
    plt.xlabel('Land Cover Type')
    plt.ylabel('Count')
    plt.title('Land Cover Distribution')
    plt.tight_layout()
    plt.savefig('../outputs/figures/land_cover_distribution.png', dpi=300, bbox_inches='tight')
    plt.show()

## 5. Create Spatial Clusters

In [ ]:
# Create spatial clusters using K-means
if 'latitude' in train.columns and 'longitude' in train.columns:
    train = create_spatial_clusters(train, n_clusters=15)
    test = create_spatial_clusters(test, n_clusters=15)
    print("\nSpatial clusters created")

In [ ]:
# Visualize spatial clusters
if all(col in train.columns for col in ['latitude', 'longitude', 'spatial_cluster']):
    plt.figure(figsize=(12, 8))
    scatter = plt.scatter(
        train['longitude'],
        train['latitude'],
        c=train['spatial_cluster'],
        cmap='tab20',
        s=5,
        alpha=0.5
    )
    plt.colorbar(scatter, label='Cluster ID')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title('Spatial Clusters')
    plt.tight_layout()
    plt.savefig('../outputs/figures/spatial_clusters.png', dpi=300, bbox_inches='tight')
    plt.show()

## 6. Create Distance Features

In [ ]:
# Define reference points (e.g., major cities, pollution sources)
# These should be replaced with actual domain-specific locations
reference_points = [
    (40.7128, -74.0060),   # Example: New York
    (34.0522, -118.2437),  # Example: Los Angeles
    (41.8781, -87.6298),   # Example: Chicago
]

if 'latitude' in train.columns and 'longitude' in train.columns:
    train = create_distance_features(train, reference_points)
    test = create_distance_features(test, reference_points)
    print("\nDistance features created")

## 7. Create Watershed Features

In [ ]:
# Create grid-based watershed assignments
if 'latitude' in train.columns and 'longitude' in train.columns:
    train = create_watershed_features(train)
    test = create_watershed_features(test)
    print("\nWatershed features created")

## 8. Encode Categorical Features

In [ ]:
# One-hot encode land cover if needed for certain models
# For tree-based models like XGBoost, label encoding is sufficient
categorical_cols = ['land_cover', 'spatial_cluster', 'watershed_id']

for col in categorical_cols:
    if col in train.columns:
        # Ensure categorical type
        train[col] = train[col].astype('category')
        test[col] = test[col].astype('category')

print("Categorical features encoded")

## 9. Feature Correlations with Target

In [ ]:
# Calculate correlations with target
if 'target' in train.columns:
    geospatial_features = ['elevation', 'slope', 'aspect', 'spatial_cluster', 
                          'land_cover', 'watershed_id']
    geospatial_features = [f for f in geospatial_features if f in train.columns]
    
    correlations = train[geospatial_features + ['target']].corr()['target'].drop('target').sort_values(ascending=False)
    
    print("\nGeospatial Feature Correlations with Target:")
    print(correlations)
    
    # Plot correlations
    plt.figure(figsize=(10, 6))
    correlations.plot(kind='barh')
    plt.xlabel('Correlation with Target')
    plt.title('Geospatial Feature Correlations')
    plt.tight_layout()
    plt.savefig('../outputs/figures/geospatial_correlations.png', dpi=300, bbox_inches='tight')
    plt.show()

## 10. Save Enhanced Dataset

In [ ]:
# Save datasets with geospatial features
save_processed_data(train, '../data/processed/train_with_geospatial.parquet', format='parquet')
save_processed_data(test, '../data/processed/test_with_geospatial.parquet', format='parquet')

print(f"\nFinal training data: {train.shape}")
print(f"Final test data: {test.shape}")
print(f"\nTotal features: {train.shape[1]}")

## Summary

This notebook successfully added geospatial features:
- **Elevation, Slope, Aspect**: Topographic characteristics
- **Land Cover**: ESA WorldCover land use types
- **Spatial Clusters**: K-means geographic groupings
- **Distance Features**: Proximity to reference locations
- **Watershed Features**: Hydrological boundaries

These features capture important spatial patterns that influence water quality and will improve model performance.

Next: Build complete training and validation pipeline (Notebook 04)